In [1]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import joblib
import numpy as np
# Load data from CSV
df = pd.read_csv("sigma_slice_data.csv")

X_cols = [c for c in df.columns if c.startswith("sensor") or c.startswith("force")]
y_cols = [c for c in df.columns if c.startswith("sigma_pt_")]

X_data = df[X_cols].to_numpy()
y_data = df[y_cols].to_numpy()

slice_coords = pd.read_csv("slice_coords.csv").to_numpy()

# Data normalization
X_positions = X_data[:, 0::3]  # x-coords
Z_positions = X_data[:, 1::3]  # z-coords
sensor_values = X_data[:, 2::3]  # sensor readings

sensor_norms = np.linalg.norm(sensor_values, axis=1, keepdims=True)
sensor_values_unit = sensor_values / (sensor_norms + 1e-8)

# Rebuild X_data with normalized sensor values
X_normalized = np.empty_like(X_data)
X_normalized[:, 0::3] = X_positions
X_normalized[:, 1::3] = Z_positions
X_normalized[:, 2::3] = sensor_values_unit

# Scale normalized inputs globally
X_scaler = StandardScaler()
X_scaled = X_scaler.fit_transform(X_normalized)
y_data_unit = y_data / (sensor_norms + 1e-8)
# Original data
X_clean = X_scaled
y_clean = y_data_unit


# Generating augmented data with noise and adding it to the original dataset
num_augments = 2
X_augmented = [X_clean + np.random.normal(0, 0.02, X_clean.shape) for _ in range(num_augments)]
y_augmented = [y_clean for _ in range(num_augments)]

X_total = np.vstack([X_clean] + X_augmented)
y_total = np.vstack([y_clean] + y_augmented)

# MLP model training
regressor = MLPRegressor(
    hidden_layer_sizes=(256, 256, 256, 256),
    activation='tanh',
    solver='adam',
    alpha=1e-5,
    learning_rate='adaptive',
    learning_rate_init=5e-4,
    max_iter=10000,
    early_stopping=True,
    n_iter_no_change=100,
    random_state=42
    
)

model = regressor
model.fit(X_total, y_total)

y_unit_pred = model.predict(X_scaled) # Predicting the stress field
y_pred = y_unit_pred * sensor_norms # Rescale to original magnitude


# Evaluation
mse = mean_squared_error(y_data, y_pred)
r2 = r2_score(y_data, y_pred)

print(f"MSE: {mse:.6f}")
print(f"R² Score: {r2:.4f}")

# ---------------------------
# Save model and scalers
joblib.dump({
    "model": model,
    "X_scaler": X_scaler,
    "slice_coords": slice_coords,
    "sensor_points": sensor_points
}, "strainzz_interpolator_shape_only3.pkl")

print("Model saved to 'strainzz_interpolator_shape_only3.pkl'")

# ---------------------------
# Visualization (sample idx = 20)
idx = 200
true_field = y_data[idx]
pred_field = y_pred[idx]
x_coords, z_coords = slice_coords[:, 0], slice_coords[:, 1]

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.tricontourf(x_coords, z_coords, true_field, levels=14)
plt.title("True ε_zz field")
plt.colorbar()

plt.subplot(1, 2, 2)
plt.tricontourf(x_coords, z_coords, pred_field, levels=14)
plt.title("Predicted ε_zz field")
plt.colorbar()

plt.tight_layout()
plt.show()
print("X mean:", X_scaled.mean(), "X std:", X_scaled.std())


NameError: name 'pd' is not defined